In [2]:
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from models.CNN import create_cnn_model

In [3]:
def load(f):
    # Move up one directory when loading the data
    file_path = os.path.join('../../../', f)
    return np.load(file_path)['arr_0']

# Load the data
x_train = load('kmnist-train-imgs.npz')/ 255.0
x_test = load('kmnist-test-imgs.npz')/ 255.0
y_train = load('kmnist-train-labels.npz')
y_test = load('kmnist-test-labels.npz')
# Reshape the data for CNN input
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [4]:
# Define the input shape and number of classes
input_shape = x_train.shape[1:]  # 784 for KMNIST
num_classes = y_train.max() + 1
x_train_aug, x_val, y_train_aug, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


In [5]:
hyperparameters = {
    'num_layers': 3,
    'filters': [32, 64, 64],
    'kernel_sizes': [(3, 3), (3, 3), (3, 3)],
    'activations': ['relu', 'relu', 'relu'],
    'dense_units': 64,
    'dense_activation': 'relu',
    'pooling': 'avg',
    'batch_normalization': True
}

In [6]:
best_model_zoom = create_cnn_model(input_shape, num_classes, hyperparameters)
best_model_no_zoom = create_cnn_model(input_shape, num_classes, hyperparameters)
datagen_zoom = ImageDataGenerator(
    zoom_range=0.1 # Zoom in on the image
)
datagen_no_zoom = ImageDataGenerator()
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
datagen_val = ImageDataGenerator()
# Wrap the datagen with our custom generator
train_generator_zoom = datagen_zoom.flow(x_train_aug, y_train_aug, batch_size=32)
train_generator_no_zoom = datagen_no_zoom.flow(x_train_aug, y_train_aug, batch_size=32)
val_generator = datagen_val.flow(x_val, y_val, batch_size=32)
# Train the model with custom augmentation
history_zoom = best_model_zoom.fit(train_generator_zoom,
                        epochs=20,
                        validation_data=val_generator,  
                        verbose=1,
                        callbacks=[early_stopping])
history_no_zoom = best_model_no_zoom.fit(train_generator_no_zoom,
                        epochs=20,
                        validation_data=val_generator,
                        verbose=1,
                        callbacks=[early_stopping])

# Evaluate on the test set
test_loss_zoom, test_accuracy_zoom = best_model_zoom.evaluate(x_test, y_test, verbose=0)
test_loss_no_zoom, test_accuracy_no_zoom = best_model_no_zoom.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy with Zoom Injection: {test_accuracy_zoom:.4f}")
print(f"Test loss with Zoom Injection: {test_loss_zoom:.4f}")
print(f"Test accuracy without Zoom Injection: {test_accuracy_no_zoom:.4f}")
print(f"Test loss without Zoom Injection: {test_loss_no_zoom:.4f}")

Epoch 1/20
1688/1688 [==============================] - 15s 7ms/step - loss: 0.2087 - accuracy: 0.9423 - val_loss: 0.1137 - val_accuracy: 0.9658
Epoch 2/20
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0753 - accuracy: 0.9775 - val_loss: 0.0649 - val_accuracy: 0.9793
Epoch 3/20
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0531 - accuracy: 0.9838 - val_loss: 0.0761 - val_accuracy: 0.9757
Epoch 4/20
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0439 - accuracy: 0.9867 - val_loss: 0.0511 - val_accuracy: 0.9842
Epoch 5/20
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0341 - accuracy: 0.9894 - val_loss: 0.0619 - val_accuracy: 0.9822
Epoch 6/20
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0282 - accuracy: 0.9911 - val_loss: 0.0494 - val_accuracy: 0.9847
Epoch 7/20
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0259 - accuracy: 0.9918 - val_loss: 0.0556 - val_ac